# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

In [2]:
pwd

'/Users/rmrazack/Documents/SairaNawaz/projects/unit-projects/project-3/starter-code'

In [3]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print (df.head())

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [5]:
print (df.describe())

            admit         gre         gpa    prestige
count  397.000000  397.000000  397.000000  397.000000
mean     0.317380  587.858942    3.392242    2.488665
std      0.466044  115.717787    0.380208    0.947083
min      0.000000  220.000000    2.260000    1.000000
25%      0.000000  520.000000    3.130000    2.000000
50%      0.000000  580.000000    3.400000    2.000000
75%      1.000000  660.000000    3.670000    3.000000
max      1.000000  800.000000    4.000000    4.000000


In [22]:
print (df.std())

admit         0.466044
gre         115.717787
gpa           0.380208
prestige      0.947083
dtype: float64


In [6]:
# frequency table for prestige and whether or not someone was admitted
print (pd.crosstab(df["admit"], df["prestige"], rownames=["admit"]))

prestige  1.0  2.0  3.0  4.0
admit                       
0          28   95   93   55
1          33   53   28   12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [8]:
#returns dummy variables for prestige categories 1-4
dummy_ranks = pd.get_dummies(df["prestige"], prefix="prestige")
print (dummy_ranks.head())

   prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0             0             0             1             0
1             0             0             1             0
2             1             0             0             0
3             0             0             0             1
4             0             0             0             1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: When modeling, we enter (n-1) categories for dummy variables. In this case, we can drop the lowest prestige category (when prestige=1), and include the other three categories in the model

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [9]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1.0':])
print (handCalc.head())

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


In [11]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
print (pd.crosstab(df["admit"], df["prestige"], rownames=["admit"]))

prestige  1.0  2.0  3.0  4.0
admit                       
0          28   95   93   55
1          33   53   28   12


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [ ]:
# the odds of being admitted if #1 ranked college: 33/61 students in a #1 ranked college are admitted. This means
# that the odds of admission is 33:28 - the ratio of admission vs non-admission for #1 ranked school

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [33]:
# the proportion for admission among non #1 ranked schools is (93)/(336)). This means that the odds of admission for 
# other schools is 93:243

#### 3.3 Calculate the odds ratio

In [34]:
((33/28)/(93/243))

3.079493087557604

#### 3.4 Write this finding in a sentenance: 

The odds of admission for #1 ranked schools is 3 times the odds of admission for other schools

#### 3.5 Print the cross tab for prestige_4

In [12]:
print (pd.crosstab(df["admit"], df["prestige"], rownames=["admit"]))

prestige  1.0  2.0  3.0  4.0
admit                       
0          28   95   93   55
1          33   53   28   12


#### 3.6 Calculate the OR 

In [41]:
# the odds for admission for colleges in Prestige_4 is 12:55. The proporition for admission for 
# attending other schools (ie prestige of 1-3) is 114/330, which means the odds is 114:216. The odds ratio is thus 0.4
((12/55)/(114/216))

0.4133971291866028

#### 3.7 Write this finding in a sentence

Answer: The odds of admission for students attending a school at prestige level 4 is 0.4 times that of students attending schools of prestige levels 1 to 3. 

## Part 4. Analysis

In [13]:
# create a clean data frame for the regression, which only includes dummy variables 2-4 (prestige=1 acts as a baseline)
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print (data.head())

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [14]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [17]:
#includes all columns except for admit 
train_cols = data.columns[1:]
train_cols

Index(['gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0',
       'intercept'],
      dtype='object')

#### 4.2 Fit the model

In [19]:
#create a logit model with admit as the outcome, and all other variables (gre, gpa, prestige 2-4) as predictors
logit = sm.Logit(data["admit"], data[train_cols])

In [20]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [50]:
# given that it is stats models it includes an intercept and the coefficients for gre, gpa and prestige
print (result.summary())

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sun, 08 Jan 2017   Pseudo R-squ.:                 0.08166
Time:                        10:21:04   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [21]:
# printing the odds ratios
print (np.exp(result.params))

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


In [22]:
# printing the confidence intervals
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.colums = ['OR', '2.5%', '97.5%']
print (np.exp(conf))

                     0         1        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: The odds of admission for someone who attended a school with Prestige ranking of #2 is 0.5 times the odds of that for someone in the #1 ranked college.  This finding is significant because the confidence interval does not contain 1.

#### 4.6 Interpret the OR of GPA

Answer: The odds of someone attending graduate school at a particular gpa is 2 x that compared to someone with a one gpa of one unit less. This odds ratio is significant because the confidence interval for the odds ratio does not include 1. 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [68]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [71]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print (gres)
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print (gpas)
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [70]:
# enumerate all possibilities
# Note - I cannot get this function to work!
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

/Users/rmrazack/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:42: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


NameError: name 'xrange' is not defined

#### 5.1 Recreate the dummy variables

In [ ]:
# recreate the dummy variables
combos.columns = ["gre", "gpa", "prestige", "intercept"] 
dummy_ranks = pd.get_dummies(combos['prestige', prefix='prestige'])
dummy_ranks.columns = ["prestige_1","prestige_2","prestige_3", "prestige_4"]
# keep only what we need for making predictions
cols_to_keep =["gre", "gpa", "prestige", "intercept"]
combos = combos[cols_to_keep].join[dummy_ranks.ix[:, "prestige_2"]]

#### 5.2 Make predictions on the enumerated dataset

In [ ]:
combos["admit_pred"] = result.predict(combos[train_cols])
print (combos.head())

#### 5.3 Interpret findings for the last 4 observations

Answer: The last four observations indicate that prestige influences the probability for admission. Individuals who attend higher prestige schools are more likely to be admitted (or have a higher probability of being admitted) if all other things are equal (gre/gpa constant)

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.